<a href="https://colab.research.google.com/github/santoshkkashyap25/QuesAns/blob/master/QuestionAnswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/MyDrive/PROJECTS/Question Answering'

Mounted at /content/drive


In [ ]:
# === Setup cell for Google Colab ===
!pip install numpy==1.24.4
!pip install -U gensim
!pip install pdfplumber
!pip install transformers

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pdfplumber
import gensim
import numpy
import torch
import nltk
from nltk.corpus import stopwords
import re

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 59.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.3 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incomp

In [ ]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

In [ ]:
pdf_path='/content/drive/MyDrive/PROJECTS/Question Answering/Documents/DI.pdf'
with pdfplumber.open(pdf_path) as pdf:
    pdf_txt = ''
    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        pdf_txt += page.extract_text()

In [ ]:
tokens = nltk.sent_tokenize(pdf_txt)
for t in tokens:
    print(t, "\n")

www.ijcrt.org © 2018 IJCRT | Volume 6, Issue 1 February 2018 | ISSN: 2320-2882
DIGITAL INDIA
VANDNA
ASSISTANT PROFESSOR
RC KHARAL BPSMV
ABSTRACT
Digital India is the beginning of digital revolution. 

It is a dream which is created by the Government of
India to ensure that government services are made available to citizens electronically , even in remote
areas , by improving online infrastructure and by increasing Internet connectivity. 

The program have one
mission and one target that is to take nation forward digitally and economically. 

The initiative will enable
people to get engaged in the innovation process which is needed by the economy to move forward. 

But to
implement this is a great challenge. 

There are many roadblocks in the way of its successful implementation
like digital illiteracy ,poor infrastructure ,low internet speed, lack of coordination among various
departments, issue pertaining to taxation etc . 

These challenges need to be addressed in order to realize th

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
import string

def clean_sentence(sentence, stopwords=False):
    sentence = sentence.lower().strip()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    if stopwords:
        sentence = remove_stopwords(sentence)
    return sentence


def get_cleaned_sentences(tokens, stopwords=False):
    return [clean_sentence(sentence, stopwords) for sentence in tokens]

In [ ]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)

['wwwijcrtorg © 2018 ijcrt volume 6 issue 1 february 2018 issn 23202882 digital india vandna assistant professor rc kharal bpsmv abstract digital india beginning digital revolution', 'dream created government india ensure government services available citizens electronically remote areas improving online infrastructure increasing internet connectivity', 'program mission target nation forward digitally economically', 'initiative enable people engaged innovation process needed economy forward', 'implement great challenge', 'roadblocks way successful implementation like digital illiteracy poor infrastructure low internet speed lack coordination departments issue pertaining taxation', 'challenges need addressed order realize potential program', 'requires lot efforts dedication departments government private sector', 'implemented properly open new opportunities citizens country introduction campaign launched government india named digital india campaign improve online framework improving in

# cosine similiarity

In [ ]:
sentences = cleaned_sentences_with_stopwords
sentence_words = [[word for word in document.split()]
                  for document in sentences]

from gensim import corpora
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
  print(key, ' : ', value)

0  :  1
1  :  2018
2  :  23202882
3  :  6
4  :  abstract
5  :  assistant
6  :  beginning
7  :  bpsmv
8  :  digital
9  :  february
10  :  ijcrt
11  :  india
12  :  is
13  :  issn
14  :  issue
15  :  kharal
16  :  of
17  :  professor
18  :  rc
19  :  revolution
20  :  the
21  :  vandna
22  :  volume
23  :  wwwijcrtorg
24  :  a
25  :  and
26  :  are
27  :  areas
28  :  available
29  :  by
30  :  citizens
31  :  connectivity
32  :  created
33  :  dream
34  :  electronically
35  :  ensure
36  :  even
37  :  government
38  :  improving
39  :  in
40  :  increasing
41  :  infrastructure
42  :  internet
43  :  it
44  :  made
45  :  online
46  :  remote
47  :  services
48  :  that
49  :  to
50  :  which
51  :  digitally
52  :  economically
53  :  forward
54  :  have
55  :  mission
56  :  nation
57  :  one
58  :  program
59  :  take
60  :  target
61  :  economy
62  :  enable
63  :  engaged
64  :  get
65  :  initiative
66  :  innovation
67  :  move
68  :  needed
69  :  people
70  :  process
71  : 

In [ ]:
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent, embedding in zip(sentences, bow_corpus):
  print(sent)
  print(embedding)

wwwijcrtorg  2018 ijcrt  volume 6 issue 1 february 2018  issn 23202882
digital india
vandna
assistant professor
rc kharal bpsmv
abstract
digital india is the beginning of digital revolution
[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]
it is a dream which is created by the government of
india to ensure that government services are made available to citizens electronically  even in remote
areas  by improving online infrastructure and by increasing internet connectivity
[(11, 1), (12, 2), (16, 1), (20, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 3), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1)]
the program have one
mission and one target that is to take nation forward digitally a

In [ ]:
user_question = "When was digital india launched ?"
question = clean_sentence(user_question, stopwords=False)
question_embedding = dictionary.doc2bow(question.split())
print(user_question, "\n", question_embedding)

When was digital india launched ? 
 [(8, 1), (11, 1), (128, 1), (165, 1), (396, 1)]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    print(index, sim, sentences[index])
    if sim > max_sim:
      max_sim = sim
      index_sim = index

  return index_sim
index = retrieveAndPrintFAQAnswer(question_embedding, bow_corpus, sentences)

0 0.12403473458920847 wwwijcrtorg  2018 ijcrt  volume 6 issue 1 february 2018  issn 23202882
digital india
vandna
assistant professor
rc kharal bpsmv
abstract
digital india is the beginning of digital revolution
1 0.9994323735785593 it is a dream which is created by the government of
india to ensure that government services are made available to citizens electronically  even in remote
areas  by improving online infrastructure and by increasing internet connectivity
2 0.9991508330917331 the program have one
mission and one target that is to take nation forward digitally and economically
3 0.9991508330917331 the initiative will enable
people to get engaged in the innovation process which is needed by the economy to move forward
4 0.9991508330917331 but to
implement this is a great challenge
5 1.0000000000000002 there are many roadblocks in the way of its successful implementation
like digital illiteracy poor infrastructure low internet speed lack of coordination among various
departments

In [ ]:
print("Question: ", question)
print("Answer: ", sentences[index])

Question:  when was digital india launched 
Answer:  there are many roadblocks in the way of its successful implementation
like digital illiteracy poor infrastructure low internet speed lack of coordination among various
departments issue pertaining to taxation etc 


# tf-idf vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

stop_words = stopwords.words('english')

# Tokenize and preprocess the sentences
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cleaned_sentences)

# Convert the user question into a TF-IDF vector
user_question ="When was digital india launched ?"
cleaned_user_question = ' '.join([word for word in nltk.word_tokenize(user_question) if word.isalnum() and word not in stop_words])
user_question_vector = vectorizer.transform([cleaned_user_question])

similarity_scores = cosine_similarity(user_question_vector, tfidf_matrix)

# Find the index of the most similar sentence
index_sim = similarity_scores.argmax()

print(user_question)
print(cleaned_sentences[index_sim])

When was digital india launched ?
aims digital india run government india digital india campaign launched empower country digitally


#  Word2vec Model

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = None
try:
  v2w_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/PROJECTS/Question Answering/w2vecmodel.mod')
  print("w2v Model Successfully loaded")
except:
  v2w_model = api.load('word2vec-google-news-300')
  v2w_model.save("/content/drive/MyDrive/PROJECTS/Question Answering/w2vecmodel.mod")
  print("w2v Model Saved")

w2vec_embedding_size = len(v2w_model['pc'])  # random word

w2v Model Successfully loaded


In [ ]:
def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec) # embedding vector


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1) # phrase embedding vector

In [ ]:
sent_embeddings = []
for sent in sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

question_embedding = getPhraseEmbedding(question, v2w_model)
index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.7304658022797411 wwwijcrtorg  2018 ijcrt  volume 6 issue 1 february 2018  issn 23202882
digital india
vandna
assistant professor
rc kharal bpsmv
abstract
digital india is the beginning of digital revolution
1 0.5642618250514614 it is a dream which is created by the government of
india to ensure that government services are made available to citizens electronically  even in remote
areas  by improving online infrastructure and by increasing internet connectivity
2 0.4298040872201784 the program have one
mission and one target that is to take nation forward digitally and economically
3 0.4201907974925139 the initiative will enable
people to get engaged in the innovation process which is needed by the economy to move forward
4 0.3419821387946181 but to
implement this is a great challenge
5 0.44204244067841825 there are many roadblocks in the way of its successful implementation
like digital illiteracy poor infrastructure low internet speed lack of coordination among various
departments

In [ ]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  when was digital india launched 
Answer:  aims of digital india
run by the government of india digital india is a campaign launched to empower the country digitally


# Glove Embedding Approach

In [ ]:
glove_model = None
try:
  glove_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/PROJECTS/Question Answering/glovemodel.mod')
  print("Glove Model Successfully loaded")
except:
  glove_model = api.load('glove-twitter-25')
  glove_model.save("/content/drive/MyDrive/PROJECTS/Question Answering/glovemodel.mod")
  print("Glove Model Saved")

glove_embedding_size = len(glove_model['pc'])

Glove Model Successfully loaded


In [ ]:
sent_embeddings = []
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, glove_model))

question_embedding = getPhraseEmbedding(question, glove_model)
retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.9050141341802046 wwwijcrtorg  2018 ijcrt  volume 6 issue 1 february 2018  issn 23202882
digital india
vandna
assistant professor
rc kharal bpsmv
abstract
digital india is the beginning of digital revolution
1 0.8714330314828446 it is a dream which is created by the government of
india to ensure that government services are made available to citizens electronically  even in remote
areas  by improving online infrastructure and by increasing internet connectivity
2 0.8825863750596621 the program have one
mission and one target that is to take nation forward digitally and economically
3 0.8658101774447189 the initiative will enable
people to get engaged in the innovation process which is needed by the economy to move forward
4 0.869156146536709 but to
implement this is a great challenge
5 0.8546417106874171 there are many roadblocks in the way of its successful implementation
like digital illiteracy poor infrastructure low internet speed lack of coordination among various
departments i

27

In [ ]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  when was digital india launched 
Answer:  aims of digital india
run by the government of india digital india is a campaign launched to empower the country digitally


# fasttext model

In [ ]:
fasttext_model = None
try:
  fasttext_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/PROJECTS/Question Answering/fasttextmodel.mod')
  print("FastText Model Successfully loaded")
except:
  fasttext_model = api.load('fasttext-wiki-news-subwords-300')
  fasttext_model.save("/content/drive/MyDrive/PROJECTS/Question Answering/fasttextmodel.mod")
  print("FastText Model Saved")

fasttext_embedding_size = len(fasttext_model['example'])

FastText Model Successfully loaded


In [ ]:
sent_embeddings = []
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, fasttext_model))

question_embedding = getPhraseEmbedding(question, fasttext_model)
retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.670302787274568 wwwijcrtorg  2018 ijcrt  volume 6 issue 1 february 2018  issn 23202882
digital india
vandna
assistant professor
rc kharal bpsmv
abstract
digital india is the beginning of digital revolution
1 0.7943682878834109 it is a dream which is created by the government of
india to ensure that government services are made available to citizens electronically  even in remote
areas  by improving online infrastructure and by increasing internet connectivity
2 0.7470279926878827 the program have one
mission and one target that is to take nation forward digitally and economically
3 0.7433186578724675 the initiative will enable
people to get engaged in the innovation process which is needed by the economy to move forward
4 0.648438528743582 but to
implement this is a great challenge
5 0.7355326299440985 there are many roadblocks in the way of its successful implementation
like digital illiteracy poor infrastructure low internet speed lack of coordination among various
departments is

33

In [ ]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  when was digital india launched 
Answer:  aims of digital india
run by the government of india digital india is a campaign launched to empower the country digitally


# BERT

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizerFast
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
answer_text = pdf_txt
question = "What is digital India?"

In [ ]:
# Apply the tokenizer to the input text, treating them as a text-pair.
encoded_inputs = tokenizer.encode_plus(question, answer_text, max_length=512, truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
token_type_ids = encoded_inputs['token_type_ids']
print('The input has a total of {:} tokens.'.format(len(input_ids[0])))

The input has a total of 512 tokens.


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
for token, id in zip(tokens, input_ids[0]):
    if id == tokenizer.sep_token_id:
        print('')
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
what          2,054
is            2,003
digital       3,617
india         2,634
?             1,029

[SEP]           102

www           7,479
.             1,012
i             1,045
##j           3,501
##cr         26,775
##t           2,102
.             1,012
org           8,917
©             1,075
2018          2,760
i             1,045
##j           3,501
##cr         26,775
##t           2,102
|             1,064
volume        3,872
6             1,020
,             1,010
issue         3,277
1             1,015
february      2,337
2018          2,760
|             1,064
issn         23,486
:             1,024
232          20,666
##0           2,692
-             1,011
288          24,841
##2           2,475
digital       3,617
india         2,634
van           3,158
##dna        28,911
assistant     3,353
professor     2,934
rc           22,110
k             1,047
##hara       11,077
##l           2,140
bp           17,531
##sm          6,491
##v           2,61

In [ ]:
# Calculate segment IDs
sep_index = input_ids[0].tolist().index(tokenizer.sep_token_id)
num_seg_a = sep_index + 1
num_seg_b = len(input_ids[0]) - num_seg_a
segment_ids = [0]*num_seg_a + [1]*num_seg_b
# Ensure segment IDs length matches input IDs length
assert len(segment_ids) == len(input_ids[0])


In [ ]:
with torch.no_grad():
    outputs = model(input_ids, token_type_ids=token_type_ids)

start_scores_tensor = outputs.start_logits
end_scores_tensor = outputs.end_logits

start_scores = start_scores_tensor.squeeze().detach().tolist()
end_scores = end_scores_tensor.squeeze().detach().tolist()

# Find the start and end indices of the answer span in the token list
answer_start = torch.argmax(torch.tensor(start_scores))
answer_end = torch.argmax(torch.tensor(end_scores))

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end+1]))

print('Answer: "' + answer + '"')


Answer: "the beginning of digital revolution"


In [ ]:
answer = tokens[answer_start]
# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
  # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]
print('Answer: "' + answer + '"')

Answer: "the beginning of digital revolution"


In [ ]:
# complete function
def answer_question(question, answer_text):
    # Tokenize inputs
    inputs = tokenizer.encode_plus(question, answer_text, max_length=512, truncation=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    print('The input has a total of {:} tokens.'.format(len(input_ids[0])))

    # Calculate segment IDs
    sep_index = input_ids[0].tolist().index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids[0]) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    # Ensure segment IDs length matches input IDs length
    assert len(segment_ids) == len(input_ids[0])
    # Generate answer scores using model
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=torch.tensor([segment_ids]))
        start_scores = outputs.start_logits.squeeze(dim=0)
        end_scores = outputs.end_logits.squeeze(dim=0)
    # Find answer start and end indices
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    # Convert answer token IDs to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end+1]))
    # Calculate confidence score
    score = float(torch.max(torch.softmax(start_scores, dim=0)))
    # Return answer and score
    return answer, score


In [ ]:
answer_question(question, answer_text)

The input has a total of 512 tokens.


('the beginning of digital revolution', 0.23458372056484222)

Tokenization is the process of dividing a text string into smaller units known as tokens. In the context of BERT (Bidirectional Encoder Representations from Transformers), a popular language model, it is common to tokenize text into tokens with a maximum length of 512 tokens. This limitation arises from the maximum input length that BERT can handle effectively.

To perform tokenization for BERT, a common approach is to utilize the encode_plus method provided by tokenizers, a widely used library in conjunction with BERT. This method takes the original text as input and returns a list of tokens. It also incorporates special BERT tokens like [CLS], [SEP], and [PAD], which hold specific meanings within the BERT framework.

To control the token length, various parameters can be specified. For instance, the max_length parameter can be set to 512, indicating the desired length for the tokenized encodings. If the original text exceeds this limit, truncation can be enabled to remove any excess tokens beyond the specified maximum length.

Conversely, in cases where the original text is shorter than the desired length, padding can be applied. By setting padding to "max_length," the encoder pads the tokenized encodings with padding tokens until the desired length of 512 tokens is reached.

By utilizing these parameters within the encode_plus method, the text can be effectively tokenized into smaller units, special BERT tokens can be added, and the length of the resulting tokenized encodings can be controlled to adhere to BERT's maximum input length of 512 tokens.

In [ ]:
tokens = tokenizer.encode_plus(user_question, pdf_txt, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
tokens

{'input_ids': [101, 2043, 2001, 3617, 2634, 3390, 1029, 102, 7479, 1012, 1045, 3501, 26775, 2102, 1012, 8917, 1075, 2760, 1045, 3501, 26775, 2102, 1064, 3872, 1020, 1010, 3277, 1015, 2337, 2760, 1064, 23486, 1024, 20666, 2692, 1011, 24841, 2475, 3617, 2634, 3158, 28911, 3353, 2934, 22110, 1047, 11077, 2140, 17531, 6491, 2615, 10061, 3617, 2634, 2003, 1996, 2927, 1997, 3617, 4329, 1012, 2009, 2003, 1037, 3959, 2029, 2003, 2580, 2011, 1996, 2231, 1997, 2634, 2000, 5676, 2008, 2231, 2578, 2024, 2081, 2800, 2000, 4480, 28926, 1010, 2130, 1999, 6556, 2752, 1010, 2011, 9229, 3784, 6502, 1998, 2011, 4852, 4274, 20831, 1012, 1996, 2565, 2031, 2028, 3260, 1998, 2028, 4539, 2008, 2003, 2000, 2202, 3842, 2830, 18397, 1998, 15318, 1012, 1996, 6349, 2097, 9585, 2111, 2000, 2131, 5117, 1999, 1996, 8144, 2832, 2029, 2003, 2734, 2011, 1996, 4610, 2000, 2693, 2830, 1012, 2021, 2000, 10408, 2023, 2003, 1037, 2307, 4119, 1012, 2045, 2024, 2116, 2346, 23467, 2015, 1999, 1996, 2126, 1997, 2049, 3144, 7375,

It return a dictionary containing three key-value pairs, input_ids ,
token_type_ids , and attention_mask .

In [ ]:
# Added return_tensors='pt' to return PyTorch tensors from the tokenizer (rather than Python lists).
tokens = tokenizer.encode_plus(pdf_txt, add_special_tokens=False, return_tensors='pt')
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (3114 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[7479, 1012, 1045,  ..., 1012, 8917, 6352]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)

for tensor in input_id_chunks:
  print(len(tensor))

510
510
510
510
510
510
54


***Chunk Preparation***

Now, we will divide our tokenized tensor into smaller chunks, each containing a maximum of 510 tokens. We chose this limit to leave room for the [CLS] and [SEP] tokens, as they require two additional positions.

***Splitting***

To accomplish this, we apply the split method to both our input IDs and attention mask tensors. Since we don't need the token type IDs, we can discard them. After splitting, we will have three chunks for each tensor set. It's important to note that we will need to add padding to the final chunk because it won't meet the required tensor size of 512, as specified by BERT.

***Adding [CLS] and [SEP] Tokens***

Next, we add the [CLS] (start of sequence) and [SEP] (separator) tokens. We can achieve this by using the torch.cat function, which concatenates a list of tensors. Since our tokens are already in token ID format, we can refer to the special tokens table mentioned earlier to create the token ID versions of [CLS] and [SEP]. Since we are performing this operation for multiple tensors, we will use a for-loop to concatenate the tokens individually for each chunk. Additionally, when concatenating the attention mask chunks, we use 1s instead of the values 101 and 102. This is because the attention mask doesn't contain token IDs but rather a series of 1s and 0s. In the attention mask, 0s represent the padding token locations, while [CLS] and [SEP] tokens, not being padding tokens, are represented by 1s.

***Padding***

To ensure that our tensor chunks meet the required length of 512 by BERT, we need to add padding. The first two chunks don't require any padding as they already meet this length requirement. However, the final chunks need padding. To determine if a chunk requires padding, we can use an if-statement to check the tensor length. If the tensor has fewer than 512 tokens, we can add padding using the torch.cat function. This statement should be added within the same for-loop where we added the [CLS] and [SEP] tokens.

In [ ]:
chunksize = 512

input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_id_chunks)):
    input_id_chunks[i] = torch.cat([
        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
    ])
    mask_chunks[i] = torch.cat([
        torch.tensor([1]), mask_chunks[i], torch.tensor([1])
    ])

    pad_len = chunksize - input_id_chunks[i].shape[0]
    if pad_len > 0:
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])

for chunk in input_id_chunks:
    print(chunk)

tensor([  101,  7479,  1012,  1045,  3501, 26775,  2102,  1012,  8917,  1075,
         2760,  1045,  3501, 26775,  2102,  1064,  3872,  1020,  1010,  3277,
         1015,  2337,  2760,  1064, 23486,  1024, 20666,  2692,  1011, 24841,
         2475,  3617,  2634,  3158, 28911,  3353,  2934, 22110,  1047, 11077,
         2140, 17531,  6491,  2615, 10061,  3617,  2634,  2003,  1996,  2927,
         1997,  3617,  4329,  1012,  2009,  2003,  1037,  3959,  2029,  2003,
         2580,  2011,  1996,  2231,  1997,  2634,  2000,  5676,  2008,  2231,
         2578,  2024,  2081,  2800,  2000,  4480, 28926,  1010,  2130,  1999,
         6556,  2752,  1010,  2011,  9229,  3784,  6502,  1998,  2011,  4852,
         4274, 20831,  1012,  1996,  2565,  2031,  2028,  3260,  1998,  2028,
         4539,  2008,  2003,  2000,  2202,  3842,  2830, 18397,  1998, 15318,
         1012,  1996,  6349,  2097,  9585,  2111,  2000,  2131,  5117,  1999,
         1996,  8144,  2832,  2029,  2003,  2734,  2011,  1996, 

In [ ]:
# rejoining the splited tokens
for tensor in range(len(input_id_chunks)):
  ans = answer_question(user_question, ' '.join(tokenizer.convert_ids_to_tokens(input_id_chunks[tensor])))
  print(ans)

The input has a total of 512 tokens.
('1st of july ’ 2015', 0.8967287540435791)
The input has a total of 512 tokens.
('[CLS] when was digital india launched? [SEP] [CLS] of this project in india. rani ( 2016 ) concluded that the digital india project provides a huge opportunity to use the latest technology to red # # ef # # ine india the paradigm # # s of service industry. it also pointed out that many projects may require some transformation # # al process, re # # eng # # ine # # ering, ref # # ine # # ments to achieve i # # j # # cr # # t # # 18 # # 01 # # 52 # # 1 international journal of creative research thoughts ( i # # j # # cr # # t ) www. i # # j # # cr # # t. org 69 # # 5 # # w # # w # # w. i # # j # # cr # # t. org © 2018 i # # j # # cr # # t | volume 6, issue 1 february 2018 | issn : 232 # # 0 - 288 # # 2 the desired service level objectives. mid # # ha ( 2016 ) concluded that digital india is a great plan to develop india for knowledge future but its improper implementatio

In [ ]:
def expand_split_sentences(pdf_txt):
  new_chunks = nltk.sent_tokenize(pdf_txt)
  length = len(new_chunks)

  new_df = [];
  for i in range(length):
    paragraph = ""
    for j in range(i, length):
      tmp_token = tokenizer.encode(paragraph + new_chunks[j])
      length_token = len(tmp_token)
      if length_token < 510:
        paragraph = paragraph + new_chunks[j]
      else:
        break;
    new_df.append(paragraph)
  return new_df

In [ ]:
fdi = pdf_txt

In [ ]:
question_fdi = "When was digital india launched?"

In [ ]:
max_score = 0;
final_answer = ""
new_df = expand_split_sentences(fdi)
for new_context in new_df:
  ans, score = answer_question(question_fdi, new_context)
  if score > max_score:
    max_score = score
    final_answer = ans
print(final_answer)
print(max_score)

The input has a total of 497 tokens.
The input has a total of 503 tokens.
The input has a total of 464 tokens.
The input has a total of 446 tokens.
The input has a total of 512 tokens.
The input has a total of 507 tokens.
The input has a total of 511 tokens.
The input has a total of 494 tokens.
The input has a total of 510 tokens.
The input has a total of 496 tokens.
The input has a total of 512 tokens.
The input has a total of 507 tokens.
The input has a total of 503 tokens.
The input has a total of 512 tokens.
The input has a total of 511 tokens.
The input has a total of 512 tokens.
The input has a total of 509 tokens.
The input has a total of 426 tokens.
The input has a total of 499 tokens.
The input has a total of 476 tokens.
The input has a total of 464 tokens.
The input has a total of 443 tokens.
The input has a total of 494 tokens.
The input has a total of 504 tokens.
The input has a total of 509 tokens.
The input has a total of 503 tokens.
The input has a total of 493 tokens.
T